# Controlla iscrizioni
Script that performs checks that FCI/ksport should do...

In [ ]:
import pandas as pd
import numpy as np

dir_iscrivi = "/home/nic/al/commissaire/data/Coda_164860.xls"

# Load dataset
df   = pd.read_excel(dir_iscrivi)
Nall = len(df)

# Name, and ID of the competiton
idGara, nomeGara = df["IdGara"][0],df["NomeGara"][0]
df = df.drop(columns=["IdGara", "NomeGara"])

# Categories of the enrolled riders
# cat_counts_all = df['Categoria'].value_counts()
cat_counts        = df['Categoria'][(df["Riserva"]=="NO") & (~df['Categoria'].isin(['TA2', 'TA3'])) ].value_counts()
cat_counts_ris    = df['Categoria'][(df["Riserva"]=="SI") & (~df['Categoria'].isin(['TA2', 'TA3'])) ].value_counts()
cat_table         = pd.merge(cat_counts, cat_counts_ris, left_index=True, right_index=True, how='outer')
cat_table.columns = ['', 'Ris']


# Remove teams that put leading spaces as their name, and re-sort
df['NomeSocieta'] = df['NomeSocieta'].str.lstrip()
df = df.sort_values('NomeSocieta')

# Group by teams
df_teams    = df.groupby("NomeSocieta")
teams_names = df_teams.groups.keys()
Nteams      = len(teams_names)



# teams_names = [s.replace(" ", "") for s in teams_names]

print("#"*(11+len(nomeGara)))
print(" {:d} - {:s}".format(idGara, nomeGara))
print("#"*(11+len(nomeGara))+"\n")
print("Dataset length: {:d} ({:d} teams)\n".format(Nall,Nteams))

print(cat_table)
print("TOT  {:d}   {:d}".format(cat_counts.sum(),cat_counts_ris.sum()))
# result.colums = ["e", "ris"]

In [ ]:
[print(i) for i in teams_names];

### Check 1 - REG
Riders with ("REG": registrazione atleta estero) must be authorised by the STF of the national federation.
FCI: the authorizations are uploaded on the GARS system.

In [ ]:
pd.Series([cat_counts, cat_counts_ris])
df["year_of_birth"] = pd.to_numeric(df["DataNascita"].str[-4:])
df["age"] = 2023 - df["year_of_birth"]
print(df[["NomeTesserato", "Categoria", "age", "NomeSocieta", "CodiceSocieta", "Riserva"]][df["Categoria"]=="REG"].to_markdown())

In [ ]:
print(df[["NomeTesserato", "Categoria", "age", "NomeSocieta", "Riserva"]][df["Categoria"]=="UN"].to_markdown())

### Check 2 - Age (& enrolled team directors)
In EL/U23 competition maximum age is 27 yr for italian riders and 25 yr for foreigners.

In [ ]:
older_25 = df["age"] >= 27

if sum(older_25) > 0:
    print(df[["NomeTesserato", "Categoria", "age", "NomeSocieta", "Nazionalità"]][older_25].to_markdown())

### Check 3 - Number of riders enrolled per team
In 1.19 competitions the maximum number of riders per team is set to **10** (and 50% reserves). This number becomes **9** if there are there are only two competitions (reg/naz/int) and the organizing team can have +2 riders. If the total number of enrolled riders is less than 80, each team may have up to 12 riders. From this computation first year U23 category riders are excluded.

In [ ]:
# Loop through each team and check the maximum number of riders

# Define the maximum number of riders per team based on the competition rules
max_riders                  = 10
# Define the maximum number of riders per team if there are only two competitions
max_riders_two_competitions = 9
# Define the maximum number of riders per team if the total enrolled riders is less than 80
max_riders_less_than_80     = 12
# Define the minimum year for eligibility based on the age limit of 18
min_year                    = 2003

Nlimite = max_riders_two_competitions

df_teams1 = df_teams[["NomeTesserato", "Categoria", "year_of_birth", "Nazionalità", "NomeSocieta", "Riserva"]]

i = 0
for team, data in df_teams1:
    i += 1
    print("\n(", i, ")  ############",team,"############")
    data       = data[~data['Categoria'].isin(['TA2', 'TA3'])]
    Niscritti  = len(data[data["Riserva"]=="NO"])
    Nriserve   = len(data[data["Riserva"]=="SI"])
    print("Iscritti:   {:d}".format(Niscritti))
    if Nriserve > Nlimite:
        print("            ^ WARNING")
    print("Riserve:    {:d}".format(Nriserve))
    if Nriserve > int(0.5*Niscritti):
        print("            ^ WARNING")
    data       = data.sort_values("year_of_birth").sort_values('Riserva')

    data['year_of_birth'] = data['year_of_birth'].astype(str)
    data.loc[data['year_of_birth'] == '2004', 'year_of_birth'] = '*2004'

    data.index =  pd.RangeIndex(start=1, stop=len(data)+1)
    # data.reset_index(drop=True, inplace=True)
    print(data.to_markdown())


    # eligible_riders = data[data['year_of_birth'] < min_year]
    # num_eligible_riders = len(eligible_riders)
    # if num_riders < 80:
    #     max_riders_team = max_riders_less_than_80
    # elif num_riders >= 80 and len(data[data['year_of_birth'] >= min_year]) > 0:
    #     max_riders_team = max_riders_two_competitions
    # else:
    #     max_riders_team = max_riders
    # if num_eligible_riders > max_riders_team:
    #     print(f'Team {team} has {num_eligible_riders} eligible riders, exceeding the maximum of {max_riders_team}')
    # else:
    #     print(f'Team {team} has {num_eligible_riders} eligible riders within the maximum of {max_riders_team}')

### Check 4 & Docs - Fiches
The organizer should send the IDs of each rider, to merge with the previous dataset

In [ ]:
dir_dorsale = "/home/nic/al/commissaire/data/165801_iscrittiDorsali.csv"

# Load dataset
dors   = pd.read_csv(dir_dorsale, delimiter="\t", header=None, usecols=[0,4])
dors.columns = ["Dors", "CodiceUci"]
#dors   = dors.drop([1,2,3], axis=1)

df_dors = df.merge(dors, how='outer', on='CodiceUci')
df_dors['Dors'] = df_dors['Dors'].astype(pd.Int64Dtype())


df_teams2t    = df_dors.groupby("NomeSocieta")
df_teams2 = df_teams2t[["Dors","NomeTesserato", "Categoria", "CodiceUci", "Nazionalità", "DataNascita", "NomeSocieta", "Riserva"]]
# df_teams2 = df_teams2.first().sort_values(by="NomeSocieta")
# df_names_keys = df["NomeSocieta"].unique()

In [ ]:
# df_teams2
i=0
for team, data in df_teams2:
    i += 1
    data       = data[~data['Categoria'].isin(['TA2', 'TA3'])]
    Niscritti  = len(data[data["Riserva"]=="NO"])
    Nriserve   = len(data[data["Riserva"]=="SI"])

    # print(data["NomeSocieta"])
    print("\n(", i, ")  ############",list(data["NomeSocieta"])[0],"############  (iscritti: {:d}, riserve: {:d})".format(Niscritti,Nriserve))

    data = data.rename(columns={"NomeTesserato":"Nome",
                                "Categoria":"Cat",
                                "CodiceUci":"Cod UCI",
                                "Nazionalità":"Naz",
                                "DataNascita":"Anno",
                                "NomeSocieta":"Società",
                                "Riserva":"Ris"})
    
    # data       = data.sort_values("Cat").sort_values('Ris')
    data.loc[data["Ris"]=="NO", "Ris"] = " "
    # data.loc[data["Dors"].isnull()] = " "
    data       = data.sort_values("Dors")
    nan_dors     = data["Dors"].isnull().values  #=="nan"
    data["Dors"] = data["Dors"].astype(str)
    data.loc[nan_dors, "Dors"]  = ""
    
    # data.loc[~nan_dors, "Dors"] = int()
    
    #.fillna('')#
    data['Anno'] = pd.to_numeric(data["Anno"].str[-4:])
    # data.loc[data['year_of_birth'] == '2004', 'year_of_birth'] = '*2004'

    data.index =  pd.RangeIndex(start=1, stop=len(data)+1)
    # data.reset_index(drop=True, inplace=True)
    print(data.to_markdown())

In [ ]:
print(df_teams.groups)

In [ ]:
df